In [1]:
### Solve Heat Equation

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numpy.testing as nptest
import scipy.sparse as sparse
import scipy.sparse.linalg as sparselinalg

In [2]:
def u(x,y): return x**4*y**5-17*np.sin(x*y)

In [3]:
def f(x,y): return 12*x**2*y**5+20*x**4*y**3+(x**2+y**2)*17*np.sin(x*y)

In [4]:
#generate mesh, i.e. x_i,y_i (including the boundary)
# our n ranges from 2 to 8, i.e. if n=2, we have h_x=h_y=1/4
# if h=2**(-n), then there are 2**n+1 grid points in each direction

def meshvector(n):
    h=2**(-n)
    pts=2**n-1
    x=np.zeros((pts**2,2))
    # print(len(x))
    for j in range(pts):
        for i in range(pts):
            x[pts*j+i,0]=(i+1)*h
            x[pts*j+i,1]=(j+1)*h
    return x

In [5]:
assert meshvector(5).shape == ((2**5-1)**2,2)
assert (meshvector(5)[0] == [1/2**5,1/2**5]).all() 

## Constructing the 9-Point-Stencil Operator

In [6]:
def BpartA(n):
    N=2**n-1
    data=[[-2]*(N)**2, [4]*(N)**2, [-2]*(N)**2]
    offsets = np.array([-1,0,1])
    b = sparse.dia_matrix((data, offsets),shape=(N,N)).toarray() 
    bmultiple = (b,)*(N)
    return sparse.csc_matrix(sparse.block_diag(bmultiple))

In [7]:
def tmpCpartA(n):
    N=2**n-1
    data=[[1]*(N)**2, [-2]*(N)**2, [1]*(N)**2]
    offsets = np.array([-1,0,1])
    b = sparse.dia_matrix((data, offsets),shape=(N,N)).toarray() 
    bmultiple = (b,)*(N)
    return sparse.csc_matrix(sparse.block_diag(bmultiple))

In [8]:
def generate_9_point_stencil(n):
    N = 2**n-1
    h=1/2**n
    C_tmp = tmpCpartA(n)
    null = sparse.csc_matrix((N**2,N))
    C_upper = sparse.bmat([[null,C_tmp[:,:-N]]])
    C_lower = sparse.bmat([[C_tmp[:,N:],null]])
    return 1/h**4 * (BpartA(n)+C_upper + C_lower)

## Creating the 5 Point Stencil operator

In [9]:
# generating B part of A

def Bpart_5_point_stencil(n):
    h=1/2**n
    N=2**n-1
    data=[[-1/h**2]*(N)**2, [4/h**2]*(N)**2, [-1/h**2]*(N)**2]
    offsets = np.array([-1,0,1])
    b = sparse.dia_matrix((data, offsets),shape=(N,N)).toarray() 
    bmultiple = (b,)*(N)
    return sparse.csc_matrix(sparse.block_diag(bmultiple))

In [10]:
def Cpart_5_point_stencil(n):
    h=1/2**n
    N=2**n-1
    cdata=[[-1/h**2]*(N)**2,[-1/h**2]*(N)**2]
    offset=np.array([N,-N])
    return sparse.csc_matrix(sparse.dia_matrix((cdata, offset),shape=((N)*(N),(N)*(N))))

In [11]:
def generate_5_point_stencil(n):
    A=Bpart_5_point_stencil(n)+Cpart_5_point_stencil(n)
    #print(A.shape)
    return A

## Creating the right hand site

In [12]:
def fgrid(n):
    N = 2**n-1
    h = 2**(-n)
    x = meshvector(n)
    f_g = f(x[:,0],x[:,1])
    L = generate_5_point_stencil(n)
    f_boundary_points = np.zeros(N**2)
    f_boundary_points[:N] = - 1/12 * f(x[:N,0],0)
    f_boundary_points[-N:] = - 1/12 * f(x[-N:,0],1)
    f_boundary_points[::N] = - 1/12 * f(0,x[::N,1])
    f_boundary_points[N-1::N] = - 1/12 * f(1,x[N-1::N,1])
    
    phi = f_g + h**2/12 * L.dot(f_g)+f_boundary_points
    fgrid = -phi
    prefactor = 1/(6*h**2)
    
    # Now we need to take the border points into account 
    fgrid[:N] += prefactor*(-u(x[:N,0]-h, x[:N,1]-h) + 2* u(x[:N,0],x[:N,1]-h) - u(x[:N,0]+h, x[:N,1]-h)) \
    + 1/h**2 * u(x[:N,0],x[:N,1]-h)
    fgrid[-N:] += prefactor*(-u(x[-N:,0]-h, x[-N,1]+h) + 2 * u(x[-N:,0], x[-N,1]+h) -u(x[-N:,0]+h, x[-N,1]+h)) \
    + 1/h**2 * u(x[-N:,0], x[-N,1]+h)
    fgrid[::N] += prefactor*(-u(x[::N,0]-h, x[::N,1]-h) + 2 * u(x[::N,0]-h, x[::N,1]) -u(x[::N,0]-h, x[::N,1]+h)) - 1/h**2 * u(x[::N,0]-h, x[::N,1])
    fgrid[N-1::N] += prefactor*(-u(x[N-1::N,0]+h, x[N-1::N,1]-h) + 
                              2 * u(x[N-1::N,0]+h, x[N-1::N,1]) -
                                u(x[N-1::N,0]+h, x[N-1::N,1]+h)) \
    + 1/h**2 * u(x[N-1::N,0]+h, x[N-1::N,1])
    
    # The cornerpoints are substracted 2 times
    fgrid[0]+=prefactor*u(0,0)
    fgrid[N-1]+=prefactor*u(1,0)
    fgrid[-N]+=prefactor*u(0,1)
    fgrid[-1]+=prefactor*u(1,1)
    return fgrid

In [13]:
def approximate_u(n):
    h = 2**(-n)
    lhs = generate_5_point_stencil(n) + h**2/6 * generate_9_point_stencil(n)
    rhs  = fgrid(n)
    return sparselinalg.spsolve(sparse.csr_matrix(lhs),rhs)

In [15]:
for n in range(2,9):
#n = 2
    x = meshvector(n)
    u_h = approximate_u(n)
    u_real = u(x[:,0],x[:,1])
#print(u_real)
#print(u_h)
    print('n =', n,' Error = ', np.max(abs(u_real-u_h)), ' h^4 = ', 2**(-4*n), 'h^2= ', 2**(-2*n))
#phi(x[:,0],x[:,1])

n = 2  Error =  0.0378410958259  h^4 =  0.00390625 h^2=  0.0625
n = 3  Error =  0.0141523686509  h^4 =  0.000244140625 h^2=  0.015625
n = 4  Error =  0.00440351671469  h^4 =  1.52587890625e-05 h^2=  0.00390625
n = 5  Error =  0.00122269174773  h^4 =  9.5367431640625e-07 h^2=  0.0009765625
n = 6  Error =  0.000321034061306  h^4 =  5.960464477539063e-08 h^2=  0.000244140625
n = 7  Error =  8.21198275158e-05  h^4 =  3.725290298461914e-09 h^2=  6.103515625e-05
n = 8  Error =  2.07539191894e-05  h^4 =  2.3283064365386963e-10 h^2=  1.52587890625e-05


## Convergence:

There seems to be still an error in the code, as the error converges with $O(h^2)$ and not $O(h^4)$